In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
csv = r"../raw/Test.csv"
pd.set_option('future.no_silent_downcasting', True)

In [7]:
def check_col(name, data):
    types = []
    count = {}
    for val in data[name]:
        if val not in types:
            types.append(val)
            count[val] = 1
        else:   
            count[val] += 1
    return types

In [17]:
df = pd.read_csv(csv)
to_drop = ["tenure", "OnlineSecurity", "OnlineBackup", "DeviceProtection","Contract", "PaperlessBilling", "PaymentMethod"]
df = df.drop(to_drop, axis=1)
df["MultipleLines"] = df["MultipleLines"].replace({"No": 0.7, "No phone service": 0})
df = df.replace({"Yes": 1, "No": 0, "Male": 1, "Female": 0})
types = check_col("InternetService", df)
df = df.replace({types[0]: 1, types[1]: 1})

# check for duplicates
df.isna().sum()

df

,customerID,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,TechSupport,StreamingTV,StreamingMovies,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,1,0,0,0,1,0,0,0,29.85,29.85,0
1,5575-GNVDE,1,0,0,0,1,0.7,1,0,0,0,56.95,1889.5,0
2,3668-QPYBK,1,0,0,0,1,0.7,1,0,0,0,53.85,108.15,1
3,7795-CFOCW,1,0,0,0,0,0,1,1,0,0,42.30,1840.75,0
4,9237-HQITU,0,0,0,0,1,0.7,1,0,0,0,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,1,0,1,1,1,1,1,1,1,1,84.80,1990.5,0
7039,2234-XADUH,0,0,1,1,1,1,1,0,1,1,103.20,7362.9,0
7040,4801-JZAZL,0,0,1,1,0,0,1,0,0,0,29.60,346.45,0
7041,8361-LTMKD,1,1,1,0,1,1,1,0,0,0,74.40,306.6,1


In [15]:
x = df.iloc[:, [1,2,3,4]].values
y = df.iloc[:, [5:]].values

SyntaxError: invalid syntax (4026040095.py, line 2)